# Интеллектуальные методы обработки видео

# Ноутбук разделен на два задания:
   * ### [Эвристический SCD](#first)
   * ### [SCD с ML](#second)
   
Сроки выполнения для каждого задания — одна неделя. Оцениваются задания независимо.

## Задание 1. Scene Change Detector
<a id='first'></a>

### Обязательно к прочтению

**Внимание!**

Opencv содержит очень много высокоуровневых функций обработки изображений (например, некоторые алгоритмы компенсации движения, отслеживания объектов, распознавания образов). Использование данной библиотеки в данном задании ограничивается:
* считыванием входного видео
* преобразованием его кадров в другие цветовые пространства
* использованием свёрток Собеля

Использовать библиотеку numpy можно без ограничений.

Если вы хотите использовать функции обработки изображений и видео из другой библиотеки, то оговорите использование этой функции в чате курса.

### Описание входных данных

Выборка для тренировки лежит https://titan.gml-team.ru:5003/sharing/yX8enupJV

Данные о каждом видео лежат в файле *train_dataset\info.json*. Это список из словарей, каждый словарь содержит информацию о расположении видео, о расположении ответов на смены сцен и содержит длину видео

In [1]:
import numpy as np
import cv2 # Для установки opencv воспользуйтесь командой в терминале conda install -c conda-forge opencv
from tqdm import tqdm_notebook as tqdm
from tqdm import notebook 
import matplotlib.pyplot as plt
import seaborn as sns
import os

%matplotlib inline

In [2]:
import json
def load_json_from_file(filename):
    with open(filename, "r") as f:
        return json.load(f, strict=False)


def dump_json_to_file(obj, filename, **kwargs):
    with open(filename, "w") as f:
        json.dump(obj, f, **kwargs)

In [3]:
video_dataset = load_json_from_file('train_dataset/info.json')
video_dataset

[{'source': 'video/03.mp4', 'scene_change': 'gt/03.json', 'len': 3250},
 {'source': 'video/04.mp4', 'scene_change': 'gt/04.json', 'len': 3392},
 {'source': 'video/05.mp4', 'scene_change': 'gt/05.json', 'len': 5662},
 {'source': 'video/07.mp4', 'scene_change': 'gt/07.json', 'len': 3321},
 {'source': 'video/08.mp4', 'scene_change': 'gt/08.json', 'len': 3396},
 {'source': 'video/10.mp4', 'scene_change': 'gt/10.json', 'len': 6096},
 {'source': 'video/14.mp4', 'scene_change': 'gt/14.json', 'len': 2326},
 {'source': 'video/17.mp4', 'scene_change': 'gt/17.json', 'len': 2904},
 {'source': 'video/21.mp4', 'scene_change': 'gt/21.json', 'len': 4898},
 {'source': 'video/22.mp4', 'scene_change': 'gt/22.json', 'len': 7749}]

### Загрузка видео ###

Загрузка видео осуществляется при помощи cv2.VideoCapture. Этот код изменять и дописывать не нужно.

In [4]:
def read_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==False:
            break
        yield frame
    cap.release()

In [10]:
frames = read_video(os.path.join('train_dataset', 'video', '03.mp4'))

Что такое frames? Это итератор на кадры видео. Чтобы пройтись по всем кадрам последовательности, воспользуйтесь следующей конструкцией:
*Аккуратно, по одной переменной frames можно пройти только один раз!*

In [ ]:
for frame in tqdm(frames):
    pass
for frame in tqdm(frames): # Второй раз уже не будет итерации
    pass

### Пишем свой простой детектор смен сцен

На данном этапе предлагается написать простой Scene Change Detector (SCD) на основе выделения характеристик кадров, подсчёта разницы между кадрами на основе данных характеристик, а также подобрать наиболее оптимальный порог для этих признаков и совместить эти признаки.
Сменой сцен в данной задаче являются только обычные мгновенные смены сцен, без дополнительных эффектов.

В качестве примера приведён простой детектор смен, который считает межкадровую разницу между кадрами.

*Важное замечание. Здесь и далее результатом алгоритма детектора сцен являются **индексы кадров начал сцен**, при этом кадры **нумеруются с 0**. Нулевой кадр в качестве ответа указывать не нужно*

<img src="Hard_cut.jpg">

In [38]:
def baseline_scene_change_detector(frames, threshold=2000, with_vis=False):
    """
    Baseline SCD

    Arguments:
    frames -- iterator on video frames
    threshold -- parameter of your algorithm (optional)
    with_vis -- saving neighboring frames at a scene change (optional)

    Returns:
    scene_changes -- list of scene changes (idx of frames)
    vis -- list of neighboring frames at a scene change (for visualization)
    metric_values -- list of metric values (for visualization)
    """
    
    def pixel_metric(frame, prev_frame):
        # Базовое расстояние между кадрами - среднеквадратическая ошибка между ними
        return np.mean((frame.astype(np.int32) - prev_frame) ** 2)

    scene_changes = []
    vis = []
    metric_values = []
    prev_frame = None
    for idx, frame in notebook.tqdm(enumerate(frames), leave=False):
        # frame - это кадр
        # idx - это номер кадра
        if prev_frame is not None:
            # Находим расстояние между соседними кадрами
            metric_value = pixel_metric(frame, prev_frame)
            metric_values.append({'metric_value' : metric_value})
        else:
            metric_values.append({'metric_value' : 0})
        prev_frame = frame
    return scene_changes, vis, metric_values

In [ ]:
frames = read_video(os.path.join('train_dataset', 'video', '03.mp4'))
cuts_base = load_json_from_file(os.path.join('train_dataset', 'gt', '03.json'))['cut']
scene_changes_base, vis_base, metric_values_base = baseline_scene_change_detector(frames, with_vis=True)

Посмотрим визуально, насколько сильно алгоритм ошибается, а также на значения метрики

In [6]:
def visualize_metric_error(frame, prev_frame, value):
    fig = plt.figure(figsize=(16,4))
    plt.suptitle('Значение метрики на текущем кадре: {:.4f}'.format(value), fontsize=24)
    ax = fig.add_subplot(1, 2, 1)
    ax.imshow(prev_frame[:,:,::-1])
    ax.set_title("Предыдущий кадр", fontsize=18)
    ax.set_xticks([])
    ax.set_yticks([])
    ax = fig.add_subplot(1, 2, 2)
    ax.imshow(frame[:,:,::-1])
    ax.set_title("Текущий кадр", fontsize=18)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.subplots_adjust(top=0.80)

In [ ]:
idx = 1
visualize_metric_error(vis_base[idx][0], vis_base[idx][1], metric_values_base[scene_changes_base[idx]])
# смена сцен

In [ ]:
idx = 10
visualize_metric_error(vis_base[idx][0], vis_base[idx][1], metric_values_base[scene_changes_base[idx]])
# ошибается, это не смена сцен

In [7]:
def visualize_metric_values(metric_values, threshold, cuts = None):
    sns.set()
    plt.figure(figsize=(16, 8))
    plt.plot(metric_values, label='Значение метрики на кадрах')
    plt.xlabel('Номер кадра')
    plt.ylabel('Значение метрики')
    plt.hlines(y=threshold, xmin=0, xmax=len(metric_values), linewidth=2, color='r', label='Пороговое значение')
    
    if cuts is not None:
        for cut in cuts:
            plt.axvline(x=cut, color='k', linestyle=':', linewidth=0.5, label='Смена сцены')

    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    plt.show()

In [ ]:
visualize_metric_values(metric_values_base, 2000, cuts_base)

**Как видим, очень плохо подобран порог, да и сам признак, похоже, сильно зашумлён. Попробуйте что-то своё!**

### Ваше решение

* В качестве решения вы должны прикрепить функцию ниже. Все пороги должны быть указаны внутри функции.  
Т.е. должен быть возможен вызов:  
`scene_changes, vis, metric_values = scene_change_detector(frames)`  
* Строку (# GRADED CELL: [function name]) менять **нельзя**. Она будет использоваться при проверке вашего решения.
* Ячейка должна содержать только **одну** функцию.

In [44]:
# GRADED CELL: scene_change_detector

def scene_change_detector(frames, threshold=None, with_vis=False):
    scene_changes = []
    vis = []
    metric_values = []
    hist_metric_values = []

    ### START CODE HERE ###
    # Ваши внешние переменные

    mask = None

    prev_frame = None
    prev_frame_hist = None
    hist_size = 16
    frames_seq_len = 9
    metrics_weight = 1.15
    hist_weight = 1.3
    hard_metrics_threshold = 11000

    def pixel_metric(frame, prev_frame):
        return np.mean((frame.astype(np.int32) - prev_frame) ** 2)
    def hist_metric(frame_hist,prev_frame_hist):
        return np.mean((frame_hist - prev_frame_hist) ** 2)
    def hist(frame):
        return cv2.calcHist(frame,[0], mask, [hist_size], [0, 256])

    ###  END CODE HERE  ###

    for idx, frame in tqdm(enumerate(frames), leave=False):
        # frame - это кадр
        # idx - это номер кадра
        if mask is not None:
            mask = np.zeros(frame[0].shape[:2], np.uint8)
            height, width, _ = frame.shape
            mask[(height // 5):(4 * height // 5), (width // 5):(4 * width // 5)] = 255

        ### START CODE HERE ###
        pass
        # Основная часть вашего алгоритма
        frame_hist = hist(frame)
        if prev_frame is not None:
            metric_value = pixel_metric(frame, prev_frame)
            hist_metric_value = hist_metric(frame_hist, prev_frame_hist)
            metric_values.append({'metric_value' : metric_value})
            hist_metric_values.append(hist_metric_value)

            if idx > frames_seq_len:
                mean = np.mean(metric_values[-frames_seq_len:])
                hist_mean = np.mean(hist_metric_values[-frames_seq_len:])
                if (metric_values[idx - ((frames_seq_len + 1) // 2) + 1] / mean * metrics_weight + hist_metric_values[idx - ((frames_seq_len + 1) // 2) + 1] / hist_mean * hist_weight) > 13.0:
                    scene_changes.append(idx - ((frames_seq_len + 1) // 2) + 1)
        else:
            metric_values.append({'metric_value' : 0.0})
            hist_metric_values.append(0)
        prev_frame = frame
        prev_frame_hist = frame_hist

        ###  END CODE HERE  ###

    return scene_changes, vis, metric_values

In [42]:
frames = read_video(os.path.join('train_dataset', 'video', '03.mp4'))
cuts = load_json_from_file(os.path.join('train_dataset', 'gt', '03.json'))['cut']
scene_changes, vis, metric_values = scene_change_detector(frames, with_vis=True)

C:\Users\doodo\AppData\Local\Temp\ipykernel_14676\2875675897.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, frame in tqdm(enumerate(frames), leave=False):


0it [00:00, ?it/s]

KeyboardInterrupt: 

#### Обратите внимание на скорость работы алгоритма! ####
Если вычислять признаки без циклов по пикселям, а пользоваться методами из numpy, то скорость будет не медленнее 7-8 кадров в секунду.
Например, вы можете использовать функцию `np.histogram` или `cv2.calcHist` для подсчёта гистограмм, а `cv2.Sobel` для применения оператора Собеля к кадру.

In [ ]:
#Посмотрим на найденные смены сцен
idx = 1 
visualize_metric_error(vis[idx][0], vis[idx][1], metric_values[scene_changes[idx]])

In [ ]:
#Посмотрим на значения метрики
visualize_metric_values(metric_values, 2000, cuts)

### Подсчёт метрики F1-Score

Чтобы оценивать алгоритм и научиться сравнивать несколько алгоритмов, нужна метрика качества. В данной задаче для оценки качества алгоритма используется F1-Score. Преимущества использования этой метрики к текущей постановке задачи смены сцен были рассказаны на лекции, напишем только формулы:
$$precision = \frac{tp}{tp+fp}$$
$$recall = \frac{tp}{tp+fn}$$
$$F = 2 * \frac{precision * recall}{precision+recall}$$

На всякий случай опишем как именно происходит подсчёт метрики для видео

1) Сначала из выборки удаляются все кадры, которые по разметке либо являются сложными переходами между сценами, либо помечены как сложные для анализа и разметки (например, титры/обилие компьютерной графики и т.п)


2) Затем для оставшихся кадров уже подсчитывается F1_Score

In [5]:
#Эти пять клеток кода править не нужно
def calculate_matrix(true_scd, predicted_scd, scene_len, not_to_use_frames=set()):
    tp, fp, tn, fn = 0, 0, 0, 0
    scene_len = scene_len
    for scd in predicted_scd:
        if scd in true_scd:
            tp += 1
        elif scd not in not_to_use_frames:
            fp += 1
    for scd in true_scd:
        if scd not in predicted_scd:
            fn += 1
    tn = scene_len - len(not_to_use_frames) - tp - fp - fn
    return tp, fp, tn, fn

In [6]:
def calculate_precision(tp, fp, tn, fn):
    return tp / max(1, (tp + fp))

In [7]:
def calculate_recall(tp, fp, tn, fn):
    return tp / max(1, (tp + fn))

In [8]:
def f1_score(true_scd, predicted_scd, scene_len, not_to_use_frames=set()):
    tp, fp, tn, fn = calculate_matrix(true_scd, predicted_scd, scene_len, not_to_use_frames)
    precision_score = calculate_precision(tp, fp, tn, fn)
    recall_score = calculate_recall(tp, fp, tn, fn)
    if precision_score + recall_score == 0:
        return 0
    else:
        return 2 * precision_score * recall_score / (precision_score + recall_score)

In [9]:
def f1_score_matrix(tp, fp, tn, fn):
    precision_score = calculate_precision(tp, fp, tn, fn)
    recall_score = calculate_recall(tp, fp, tn, fn)
    if precision_score + recall_score == 0:
        return 0
    else:
        return 2 * precision_score * recall_score / (precision_score + recall_score)

### Тестируем разработанный метод сразу на нескольких видео 

Проверим, насколько хорошо работает разработанный метод. *Учтите, что итоговое тестирование будет производиться на аналогичном, но недоступном вам наборе видео, но все параметры алгоритмов должны быть указаны вами (иными словами - подобраны на тренировочном наборе).*

In [10]:
def run_scene_change_detector_all_video(scene_change_detector, dataset_path):
    video_dataset = load_json_from_file(os.path.join(dataset_path, 'info.json'))
    param_log = {
        '_mean_f1_score': []
    }
    for video_info in tqdm(video_dataset, leave=False):
        # Загружаем видео, его длину и смены сцен
        frames = read_video(os.path.join(dataset_path, video_info['source']))
        video_len = video_info['len']
        true_scene_changes = load_json_from_file(os.path.join(dataset_path, video_info['scene_change']))
        
        # Составляем список сцен, которые не будут тестироваться
        not_use_frames = set()
        for type_scene_change in ['trash', 'fade', 'dissolve']:
            for bad_scene_range in true_scene_changes.get(type_scene_change, []):
                not_use_frames.update(list(range(bad_scene_range[0], bad_scene_range[1] + 1)))
        
        predicted_scene_changes, _, _ = scene_change_detector(frames)
        
        param_log['f1_score_{}'.format(video_info['source'])] = f1_score(
            true_scene_changes['cut'],
            predicted_scene_changes,
            video_len,
            not_use_frames
        )
        video_tp, video_fp, video_tn, video_fn = calculate_matrix(
            true_scene_changes['cut'],
            predicted_scene_changes,
            video_len,
            not_use_frames
        )
        
        param_log['tp_{}'.format(video_info['source'])] = video_tp
        param_log['fp_{}'.format(video_info['source'])] = video_fp
        param_log['tn_{}'.format(video_info['source'])] = video_tn
        param_log['fn_{}'.format(video_info['source'])] = video_fn 
        param_log['_mean_f1_score'].append(param_log['f1_score_{}'.format(video_info['source'])])
    param_log['_mean_f1_score'] = np.mean(param_log['_mean_f1_score'])
    return param_log

In [11]:
video_dataset = 'train_dataset'

Данная функция поможет вам посмотреть, на каких видео и на сколько ошибается ваш метод. Прогнать метод на отдельном видео и детально посмотреть кадры вы могли выше.

Кроме того, с помощью этой функции вы можете подобрать оптимальные параметры для метода.

In [ ]:
#Протестируем базовый метод
run_scene_change_detector_all_video(baseline_scene_change_detector, video_dataset)

In [ ]:
#Протестируем разработанный вами метод
run_scene_change_detector_all_video(scene_change_detector, video_dataset)

C:\Users\doodo\AppData\Local\Temp\ipykernel_14676\4126358636.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for video_info in tqdm(video_dataset, leave=False):


  0%|          | 0/10 [00:00<?, ?it/s]

C:\Users\doodo\AppData\Local\Temp\ipykernel_14676\3384297497.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, frame in tqdm(enumerate(frames), leave=False):


0it [00:00, ?it/s]

Когда вы смотрите на результат, обращайте внимание на **_mean_f1_score**  
Именно по этой метрике будет производится финальное оценивание.

## Бонусное задание: распознавание смен сцен типа "наложения"

На практике кроме катов часто встречаются смены сцен, где происходит "наложение" одной сцены на другую:

<img src="Dissolve.jpg">

### Ваше решение 

* В качестве решения вы должны прикрепить функцию ниже. Все пороги должны быть указаны внутри функции.  
Т.е. должен быть возможен вызов:  
`scene_changes, vis, metric_values = scene_change_detector_dissolve(frames)`  
* Строку (# GRADED CELL: [function name]) менять **нельзя**. Она будет использоваться при проверке вашего решения.
* Ячейка должна содержать только **одну** функцию.

In [ ]:
# GRADED CELL: scene_change_detector_dissolve

def scene_change_detector_dissolve(frames, threshold=None, with_vis=False):
    scene_changes = []
    vis = []
    metric_values = []
    
    ### START CODE HERE ###
    # Ваши внешние переменные
    ###  END CODE HERE  ###
    
    for idx, frame in tqdm(enumerate(frames), leave=False):
        # frame - это кадр
        # idx - это номер кадра
        
        ### START CODE HERE ###
        pass
        # Основная часть вашего алгоритма
        ###  END CODE HERE  ###

    return scene_changes, vis, metric_values

В качестве метрики качества используется видоизменённый f1-score:

Так как смена сцен не происходит за один кадр, попаданием считается попадание ответа смены сцен в отрезок, где происходит наложение.  
**Обратите внимание**, что несколько раз указывать одну смену сцен не нужно.

Попадание вне отрезков смен сцен путём наложения считается как false positive, не попадание в указанный отрезок - как false negative

In [ ]:
#Эти три клетки кода править не нужно
def calculate_matrix_dissolve(true_scd, predicted_scd, scene_len):
    tp, fp, tn, fn = 0, 0, 0, 0
    scene_len = scene_len
    checked_dissolve_segments = set()
    total_scene_dissolve_len = np.sum([dissolve_segment[1] - dissolve_segment[0] + 1 for dissolve_segment in true_scd])
    for scd in predicted_scd:
        for dissolve_segment in true_scd:
            if scd in range(dissolve_segment[0], dissolve_segment[1] + 1):
                tp += 1
                checked_dissolve_segments.add(tuple(dissolve_segment))
                break
        else:
            fp += 1
    fn = len(true_scd) - len(checked_dissolve_segments)
    tn = scene_len - total_scene_dissolve_len + len(true_scd) - tp - fp - fn
    return tp, fp, tn, fn

In [ ]:
def f1_score_dissolve(true_scd, predicted_scd, scene_len):
    tp, fp, tn, fn = calculate_matrix_dissolve(true_scd, predicted_scd, scene_len)
    precision_score = calculate_precision(tp, fp, tn, fn)
    recall_score = calculate_recall(tp, fp, tn, fn)
    if precision_score + recall_score == 0:
        return 0
    else:
        return 2 * precision_score * recall_score / (precision_score + recall_score)

In [ ]:
def run_scene_change_detector_all_video_dissolve(scene_change_detector, dataset_path):
    video_dataset = load_json_from_file(os.path.join(dataset_path, 'info.json'))
    param_log = {
        '_mean_f1_score': []
    }
    for video_info in tqdm(video_dataset, leave=False):
        frames = read_video(os.path.join(dataset_path, video_info['source']))
        video_len = video_info['len']
        true_scene_changes = load_json_from_file(os.path.join(dataset_path, video_info['scene_change']))
        
        predicted_scene_changes, _, _ = scene_change_detector(frames)
        param_log['f1_score_{}'.format(video_info['source'])] = f1_score_dissolve(
            true_scene_changes.get('dissolve', []),
            predicted_scene_changes,
            video_len
        )
        video_tp, video_fp, video_tn, video_fn = calculate_matrix_dissolve(
            true_scene_changes.get('dissolve', []),
            predicted_scene_changes,
            video_len
        )
        param_log['tp_{}'.format(video_info['source'])] = video_tp
        param_log['fp_{}'.format(video_info['source'])] = video_fp
        param_log['tn_{}'.format(video_info['source'])] = video_tn
        param_log['fn_{}'.format(video_info['source'])] = video_fn
        param_log['_mean_f1_score'].append(param_log['f1_score_{}'.format(video_info['source'])])
    param_log['_mean_f1_score'] = np.mean(param_log['_mean_f1_score'])
    return param_log

In [ ]:
video_dataset_path = 'train_dataset'

In [ ]:
#Протестируем разработанный вами метод
run_scene_change_detector_all_video_dissolve(scene_change_detector_dissolve, video_dataset_path)

### Немного об оценивании задания

Оценивание задания будет производиться по следующей схеме:  

Пусть на скрытой выборке по F-метрике вы получили X, лучшее решение получило Y.

1. Базовая часть оценивется как $$20 * \left(\frac{\max(0, X_{base} - 0.5)}{Y_{base} - 0.5}\right)^2 + Bonus_{base}$$ Бонусные баллы $Bonus$ можно получить за оригинальные идеи в задаче или в её реализации
2. Дополнительное задание оценивается как $$5 * \frac{\max(0, X_{add} - 0.1)}{Y_{add} - 0.1} + Bonus_{add}$$Процесс получения бонусных баллов аналогичен получению бонусных баллов в базовой части

### Ваши ощущения ##

*До дедлайна пару часов и вы никак не можете улучшить текущее решение? Или наоборот, вы всё сделали очень быстро? Опишите кратко ваши ощущения от задания - сколько времени вы потратили на задание, сколько вы потратили на изучение питона и установку необходимых библиотек, как быстро вы придумывали новые идеи и как они давали прирост по метрике и в целом насколько это задание вам понравилось и что хотели бы изменить/добавить.*

<a id='second'></a>

## Задание 2. Scene change detector. Машинное обучение

**Внимание!**

В этом задании можно использовать все, что разрешалось в Задании №1, а также библиотеки:
* pandas
* sklearn

Большинство функций, использующихся в этом задании, реализованы выше.

### Бейзлайн

In [25]:
from sklearn.svm import SVC
import pandas as pd
import pickle

Обучим простой SVM классификатор над метрикой попиксельной разницы кадров на нескольких видео. Воспользуемся функцией из первого задания

In [41]:
def get_train_data(train_videos):
    X_train, y_train = pd.DataFrame(), np.array([])
    for video in train_videos:
        frames = read_video(os.path.join('train_dataset', 'video', f'{video}.mp4'))
        # baseline функция попиксельного сравнения кадров из прошлого задания
        # нам нужны не сами смены сцен, а только значения метрик
        _, _, metric_values = baseline_scene_change_detector(frames) 
        
        cuts = load_json_from_file(os.path.join('train_dataset', 'gt', f'{video}.json'))['cut']
        video_scenes = np.array([0 for i in range(len(metric_values))])
        video_scenes[cuts] += 1
        
        # добавляем в разметку текущее видео
        X_train = X_train.append(metric_values)
        y_train = np.hstack((y_train, video_scenes))
        
    return X_train, y_train 

In [42]:
train_videos = ['04', '05']

X_train, y_train = get_train_data(train_videos)

0it [00:00, ?it/s]

C:\Users\doodo\AppData\Local\Temp\ipykernel_13920\1715947297.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(metric_values)


0it [00:00, ?it/s]

C:\Users\doodo\AppData\Local\Temp\ipykernel_13920\1715947297.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(metric_values)


In [43]:
import pandas as pd

In [44]:
# создание модели
# подберите лучшие параметры для данной задачи
params = {"kernel": "rbf", "C": 1}
model = SVC(**params)
model.fit(pd.DataFrame(X_train), y_train)

SVC(C=1)

 Сохраним модель в файле *model.pkl*

In [45]:
pickle.dump(model, open("model.pkl", "wb"))

Посмотрим как модель работает на тестовых видео

Обратите внимание на то, что внутри функции модель загружается из памяти из файла *model.pkl*

In [46]:
def baseline_scene_change_detection_ml(frames):
    # подготавливаем данные для видео
    _, _, metric_values = baseline_scene_change_detector(frames) 
    X_test = pd.DataFrame(metric_values)
    
    # загружаем модель и делаем предсказания
    model = pickle.load(open("model.pkl", 'rb'))
    predict_cuts = model.predict(X_test)
    
    return np.where(predict_cuts > 0)[0]

In [47]:
def run_scene_change_detector_ml_one_video(scene_change_detector, dataset_path, video_num):
    video_info = load_json_from_file(os.path.join(dataset_path, 'info.json'))[video_num]
    
    # Загружаем видео, его длину и смены сцен
    frames = read_video(os.path.join(dataset_path, video_info['source']))
    video_len = video_info['len']
    true_scene_changes = load_json_from_file(os.path.join(dataset_path, video_info['scene_change']))

    # Составляем список сцен, которые не будут тестироваться
    not_use_frames = set()
    for type_scene_change in ['trash', 'fade', 'dissolve']:
        for bad_scene_range in true_scene_changes.get(type_scene_change, []):
            not_use_frames.update(list(range(bad_scene_range[0], bad_scene_range[1] + 1)))

    predicted_scene_changes = scene_change_detector(frames)

    return  f1_score(
        true_scene_changes['cut'],
        predicted_scene_changes,
        video_len,
        not_use_frames
    )
        

Посчитаем F1 score для одного видео:

In [48]:
video_num = 3
run_scene_change_detector_ml_one_video(baseline_scene_change_detection_ml, 'train_dataset', video_num)

0it [00:00, ?it/s]

0.8317757009345794

In [35]:
predicted_scene_changes

array([ 144,  170,  213,  273,  277,  278,  280,  281,  282,  284,  289,
        291,  292,  293,  294,  295,  296,  297,  298,  299,  300,  301,
        307,  308,  309,  313,  315,  355,  399,  427,  490,  529,  578,
        610,  651,  682,  739,  818,  837,  857,  880,  949,  975, 1024,
       1056, 1092, 1136, 1171, 1189, 1209, 1230, 1305, 1374, 1390, 1447,
       1480, 1500, 1515, 1533, 1547, 1576, 1604, 1622, 1664, 1686, 1703,
       1728, 1742, 1763, 1785, 1862, 1889, 1917, 1950, 1970, 1996, 2018,
       2052, 2070, 2077, 2101, 2129, 2178, 2191, 2193, 2202, 2241, 2283,
       2324, 2349, 2371, 2403, 2438, 2461, 2488, 2555, 2582, 2644, 2664,
       2712, 2751, 2786, 2851, 2870, 2883, 2899, 2912, 2954, 2991, 3020,
       3136, 3216], dtype=int64)

In [23]:
type(predicted_scene_changes)

numpy.ndarray

### Ваше решение

Чтобы использовать свою обученную модель при отправке решения, необходимо сохранить ее через пакет pickle в файл model.pkl и отправить его вместе с jupyter ноутбуком.
Этот файл вы можете открывать и использовать прямо в функции вашего решения

* В качестве решения вы должны прикрепить функцию ниже. Все пороги должны быть указаны внутри функции.  
Т.е. должен быть возможен вызов:  
`scene_changes, vis, metric_values = scene_change_detector_dissolve(frames)`  
* Строку (# GRADED CELL: [function name]) менять **нельзя**. Она будет использоваться при проверке вашего решения.
* Ячейка должна содержать только **одну** функцию.

In [2]:
# GRADED CELL: scene_change_detector_ml

def scene_change_detector_ml(frames, with_vis = False):
    def pixel_metric(frame, prev_frame):
        return np.nanmean((frame[:,:,0].astype(np.int32) - prev_frame[:,:,0]) ** 2)
    def hist_metric(frame_hist,prev_frame_hist):
        return np.nanmean(abs(frame_hist - prev_frame_hist))
    def hist(frame, mask, hist_size):
        return cv2.calcHist([frame],[0], mask, [hist_size], [0, 256])
    def sobel(frame):
        return np.where(cv2.Sobel(frame[:,:,0],cv2.CV_8U,1,0) + cv2.Sobel(frame[:,:,0],cv2.CV_8U,0,1) > 250, 1, 0)
    def sobel_metric(frame_sobel, prev_frame_sobel):
        return np.nanmean((frame_sobel - prev_frame_sobel) ** 2)
    def br_mean(frame):
        return np.mean(frame[:,:,0])
    def metric_generator(frames):
        metrics = []
        metric_values = []
        hist_metric_values = []
        hist_metric_values_1q = []
        hist_metric_values_2q = []
        hist_metric_values_3q = []
        hist_metric_values_4q = []

        ### START CODE HERE ###
        # Ваши внешние переменные

        mask = None
        mask_1q = None
        mask_2q = None
        mask_3q = None
        mask_4q = None

        prev_frame = None
        prev_frame_hist = None
        prev_frame_1q_hist = None
        prev_frame_2q_hist = None
        prev_frame_3q_hist = None
        prev_frame_4q_hist = None
        prev_frame_sobel = None

        hist_size = 256
        frames_seq_len = 7

        ###  END CODE HERE  ###

        for idx, frame in tqdm(enumerate(frames), leave=False):
            # frame - это кадр
            # idx - это номер кадра
            mean = float(0)
            hist_mean = float(0)
            frame_features = {}
            if mask is None:
                height, width, _ = frame.shape
                mask = np.zeros((height, width), np.uint8)
                mask[(2 * height // 7):(5 * height // 7), (2 * width // 7):(5 * width // 7)] = 255

                mask_1q = frame[0 : height // 2, 0 : width // 2]
                mask_2q = frame[0 : height // 2, width // 2 : width]
                mask_3q = frame[height // 2 : height, 0 : width // 2]
                mask_4q = frame[height // 2 : height, width // 2 : width]


            ### START CODE HERE ###
            frame_features['br_mean'] = br_mean(frame)
            frame_hist = hist(frame, mask, hist_size)
            frame_sobel = sobel(frame)

            frame_1q_hist = hist(mask_1q, None, hist_size)
            frame_2q_hist = hist(mask_2q, None, hist_size)
            frame_3q_hist = hist(mask_3q, None, hist_size)
            frame_4q_hist = hist(mask_4q, None, hist_size)

            if prev_frame is not None:
                frame_features['br_mean_prev'] = br_mean(prev_frame)
                metric_value = pixel_metric(frame, prev_frame)
                hist_metric_value = hist_metric(frame_hist, prev_frame_hist)
                #sobel_metric_value = sobel_metric(frame_sobel, prev_frame_sobel)
                metric_values.append(metric_value)
                hist_metric_values.append(hist_metric_value)
                frame_features['metric_value'] = metric_value
                #frame_features['sobel_metric_value'] = sobel_metric_value

                hist_metric_value_1q = hist_metric(frame_1q_hist, prev_frame_1q_hist)
                hist_metric_value_2q = hist_metric(frame_1q_hist, prev_frame_2q_hist)
                hist_metric_value_3q = hist_metric(frame_1q_hist, prev_frame_3q_hist)
                hist_metric_value_4q = hist_metric(frame_4q_hist, prev_frame_4q_hist)

                hist_metric_values_1q.append(hist_metric_value_1q)
                hist_metric_values_2q.append(hist_metric_value_2q)
                hist_metric_values_3q.append(hist_metric_value_3q)
                hist_metric_values_4q.append(hist_metric_value_4q)

                frame_features['hist_metric_value'] =  hist_metric_value
                frame_features['hist_metric_value_1q'] =  hist_metric_value_1q
                frame_features['hist_metric_value_2q'] =  hist_metric_value_2q
                frame_features['hist_metric_value_3q'] =  hist_metric_value_3q
                frame_features['hist_metric_value_4q'] =  hist_metric_value_4q

                if idx >= frames_seq_len - 1:
                    mean = float(np.nanmean(metric_values[-frames_seq_len: idx - ((frames_seq_len + 1) // 2) + 1] + metric_values[idx - ((frames_seq_len + 1) // 2) + 2:]))
                    hist_mean = float(np.nanmean(hist_metric_values[-frames_seq_len: idx - ((frames_seq_len + 1) // 2) + 1] + hist_metric_values[idx - ((frames_seq_len + 1) // 2) + 2:]))
                    metrics[idx - ((frames_seq_len + 1) // 2) + 1]['mean'] = mean
                    metrics[idx - ((frames_seq_len + 1) // 2) + 1]['hist_mean'] = hist_mean

                    hist_mean_1q = float(np.nanmean(hist_metric_values_1q[-frames_seq_len: idx - ((frames_seq_len + 1) // 2) + 1] + hist_metric_values_1q[idx - ((frames_seq_len + 1) // 2) + 2:]))
                    hist_mean_2q = float(np.nanmean(hist_metric_values_2q[-frames_seq_len: idx - ((frames_seq_len + 1) // 2) + 1] + hist_metric_values_2q[idx - ((frames_seq_len + 1) // 2) + 2:]))
                    hist_mean_3q = float(np.nanmean(hist_metric_values_3q[-frames_seq_len: idx - ((frames_seq_len + 1) // 2) + 1] + hist_metric_values_3q[idx - ((frames_seq_len + 1) // 2) + 2:]))
                    hist_mean_4q = float(np.nanmean(hist_metric_values_4q[-frames_seq_len: idx - ((frames_seq_len + 1) // 2) + 1] + hist_metric_values_4q[idx - ((frames_seq_len + 1) // 2) + 2:]))

                    metrics[idx - ((frames_seq_len + 1) // 2) + 1]['hist_mean_1q'] = hist_mean_1q
                    metrics[idx - ((frames_seq_len + 1) // 2) + 1]['hist_mean_2q'] = hist_mean_2q
                    metrics[idx - ((frames_seq_len + 1) // 2) + 1]['hist_mean_3q'] = hist_mean_3q
                    metrics[idx - ((frames_seq_len + 1) // 2) + 1]['hist_mean_4q'] = hist_mean_4q

                elif idx == frames_seq_len - 2:
                    for i in range(0, (frames_seq_len) // 2):
                        mean = float(np.nanmean(metric_values[0: i] + metric_values[i + 1 : (frames_seq_len)// 2 + i + 1]))
                        metrics[i]['mean'] = mean
                        hist_mean = float(np.nanmean(hist_metric_values[0: i] + hist_metric_values[i + 1 : (frames_seq_len)// 2 + i + 1]))
                        metrics[i]['hist_mean'] = hist_mean

                        hist_mean_1q = float(np.nanmean(hist_metric_values_1q[0: i] + hist_metric_values_1q[i + 1 : (frames_seq_len)// 2 + i + 1]))
                        hist_mean_2q = float(np.nanmean(hist_metric_values_2q[0: i] + hist_metric_values_2q[i + 1 : (frames_seq_len)// 2 + i + 1]))
                        hist_mean_3q = float(np.nanmean(hist_metric_values_3q[0: i] + hist_metric_values_3q[i + 1 : (frames_seq_len)// 2 + i + 1]))
                        hist_mean_4q = float(np.nanmean(hist_metric_values_4q[0: i] + hist_metric_values_4q[i + 1 : (frames_seq_len)// 2 + i + 1]))

                        metrics[i]['hist_mean_1q'] = hist_mean_1q
                        metrics[i]['hist_mean_2q'] = hist_mean_2q
                        metrics[i]['hist_mean_3q'] = hist_mean_3q
                        metrics[i]['hist_mean_4q'] = hist_mean_4q

            else:
                frame_features['br_mean_prev'] = 0

                frame_features['metric_value'] = 0
                metric_values.append(0)
                frame_features['hist_metric_value'] = 0
                hist_metric_values.append(0)
                #frame_features['sobel_metric_value'] = 0

                frame_features['hist_metric_value_1q'] =  0
                frame_features['hist_metric_value_2q'] =  0
                frame_features['hist_metric_value_3q'] =  0
                frame_features['hist_metric_value_4q'] =  0

                hist_metric_values_1q.append(0)
                hist_metric_values_2q.append(0)
                hist_metric_values_3q.append(0)
                hist_metric_values_4q.append(0)

            prev_frame = frame
            prev_frame_hist = frame_hist
            prev_frame_sobel = frame_sobel

            prev_frame_1q_hist = frame_1q_hist
            prev_frame_2q_hist = frame_2q_hist
            prev_frame_3q_hist = frame_3q_hist
            prev_frame_4q_hist = frame_4q_hist

            metrics.append(frame_features)
            ###  END CODE HERE  ###
        for i in range(len(metric_values) - (frames_seq_len) // 2, len(metric_values)):
            mean = float(np.nanmean(metric_values[i - (frames_seq_len) // 2 : i] + metric_values[i + 1 : len(metric_values)]))
            metrics[i]['mean'] = mean
            hist_mean = float(np.nanmean(hist_metric_values[i - (frames_seq_len) // 2 : i] + hist_metric_values[i + 1 : len(metric_values)]))
            metrics[i]['hist_mean'] = hist_mean

            hist_mean_1q = float(np.nanmean(hist_metric_values_1q[i - (frames_seq_len) // 2 : i] + hist_metric_values_1q[i + 1 : len(metric_values)]))
            hist_mean_2q = float(np.nanmean(hist_metric_values_2q[i - (frames_seq_len) // 2 : i] + hist_metric_values_2q[i + 1 : len(metric_values)]))
            hist_mean_3q = float(np.nanmean(hist_metric_values_3q[i - (frames_seq_len) // 2 : i] + hist_metric_values_3q[i + 1 : len(metric_values)]))
            hist_mean_4q = float(np.nanmean(hist_metric_values_4q[i - (frames_seq_len) // 2 : i] + hist_metric_values_4q[i + 1 : len(metric_values)]))

            metrics[i]['hist_mean_1q'] = hist_mean_1q
            metrics[i]['hist_mean_2q'] = hist_mean_2q
            metrics[i]['hist_mean_3q'] = hist_mean_3q
            metrics[i]['hist_mean_4q'] = hist_mean_4q

        return metrics

    X_test = pd.DataFrame(metric_generator(frames))
    model = pickle.load(open("model.pkl", 'rb'))
    predict_cuts = model.predict(X_test)

    return np.where(predict_cuts == 1)[0], None, None

Проверим ваше решение на всех видео.

Не забывайте о том, что при итоговой оценке решений будет использоваться другой набор видео. Не переобучите модель!

In [ ]:
video_dataset_path = 'train_dataset'

In [ ]:
run_scene_change_detector_all_video(scene_change_detector_ml, video_dataset_path)

**Советы**

* Используйте кросс-валидацию
* Подумайте как лучше разделять видео на тренировочную и тестовые выборки
* Подбирайте параметры модели (в библиотеке sklearn есть метод GridSearchCV для автоматического подбора параметров)
* Пробуйте разные методы машинного обучения (из sklearn)

## Бонусное задание: детектор смен сцен типа наложение

Аналогично детектору из задания №1 за исключением того, что можно (и нужно) использовать машинное обучение:)

In [ ]:
# GRADED CELL: scene_change_detector_dissolve_ml

def scene_change_detector_dissolve_ml(frames, threshold=None, with_vis=False):
    def pixel_metric(frame, prev_frame):
        return np.nanmean((frame[:,:,0].astype(np.int32) - prev_frame[:,:,0]) ** 2)
    def hist_metric(frame_hist,prev_frame_hist):
        return np.nanmean(abs(frame_hist - prev_frame_hist))
    def hist(frame, mask, hist_size):
        return cv2.calcHist([frame],[0], mask, [hist_size], [0, 256])
    def sobel(frame):
        return np.where(cv2.Sobel(frame[:,:,0],cv2.CV_8U,1,0) + cv2.Sobel(frame[:,:,0],cv2.CV_8U,0,1) > 250, 1, 0)
    def sobel_metric(frame_sobel, prev_frame_sobel):
        return np.nanmean((frame_sobel - prev_frame_sobel) ** 2)
    def br_mean(frame):
        return np.mean(frame[:,:,0])
    def metric_generator(frames):
        metrics = []
        metric_values = []
        hist_metric_values = []
        hist_metric_values_1q = []
        hist_metric_values_2q = []
        hist_metric_values_3q = []
        hist_metric_values_4q = []

        ### START CODE HERE ###
        # Ваши внешние переменные

        mask = None
        mask_1q = None
        mask_2q = None
        mask_3q = None
        mask_4q = None

        prev_frame = None
        prev_frame_hist = None
        prev_frame_1q_hist = None
        prev_frame_2q_hist = None
        prev_frame_3q_hist = None
        prev_frame_4q_hist = None
        prev_frame_sobel = None

        hist_size = 256
        frames_seq_len = 7

        ###  END CODE HERE  ###

        for idx, frame in tqdm(enumerate(frames), leave=False):
            # frame - это кадр
            # idx - это номер кадра
            mean = float(0)
            hist_mean = float(0)
            frame_features = {}
            if mask is None:
                height, width, _ = frame.shape
                mask = np.zeros((height, width), np.uint8)
                mask[(2 * height // 7):(5 * height // 7), (2 * width // 7):(5 * width // 7)] = 255

                mask_1q = frame[0 : height // 2, 0 : width // 2]
                mask_2q = frame[0 : height // 2, width // 2 : width]
                mask_3q = frame[height // 2 : height, 0 : width // 2]
                mask_4q = frame[height // 2 : height, width // 2 : width]


            ### START CODE HERE ###
            frame_features['br_mean'] = br_mean(frame)
            frame_hist = hist(frame, mask, hist_size)
            frame_sobel = sobel(frame)

            frame_1q_hist = hist(mask_1q, None, hist_size)
            frame_2q_hist = hist(mask_2q, None, hist_size)
            frame_3q_hist = hist(mask_3q, None, hist_size)
            frame_4q_hist = hist(mask_4q, None, hist_size)

            if prev_frame is not None:
                frame_features['br_mean_prev'] = br_mean(prev_frame)
                metric_value = pixel_metric(frame, prev_frame)
                hist_metric_value = hist_metric(frame_hist, prev_frame_hist)
                #sobel_metric_value = sobel_metric(frame_sobel, prev_frame_sobel)
                metric_values.append(metric_value)
                hist_metric_values.append(hist_metric_value)
                frame_features['metric_value'] = metric_value
                #frame_features['sobel_metric_value'] = sobel_metric_value

                hist_metric_value_1q = hist_metric(frame_1q_hist, prev_frame_1q_hist)
                hist_metric_value_2q = hist_metric(frame_1q_hist, prev_frame_2q_hist)
                hist_metric_value_3q = hist_metric(frame_1q_hist, prev_frame_3q_hist)
                hist_metric_value_4q = hist_metric(frame_4q_hist, prev_frame_4q_hist)

                hist_metric_values_1q.append(hist_metric_value_1q)
                hist_metric_values_2q.append(hist_metric_value_2q)
                hist_metric_values_3q.append(hist_metric_value_3q)
                hist_metric_values_4q.append(hist_metric_value_4q)

                frame_features['hist_metric_value'] =  hist_metric_value
                frame_features['hist_metric_value_1q'] =  hist_metric_value_1q
                frame_features['hist_metric_value_2q'] =  hist_metric_value_2q
                frame_features['hist_metric_value_3q'] =  hist_metric_value_3q
                frame_features['hist_metric_value_4q'] =  hist_metric_value_4q

                if idx >= frames_seq_len - 1:
                    mean = float(np.nanmean(metric_values[-frames_seq_len: idx - ((frames_seq_len + 1) // 2) + 1] + metric_values[idx - ((frames_seq_len + 1) // 2) + 2:]))
                    hist_mean = float(np.nanmean(hist_metric_values[-frames_seq_len: idx - ((frames_seq_len + 1) // 2) + 1] + hist_metric_values[idx - ((frames_seq_len + 1) // 2) + 2:]))
                    metrics[idx - ((frames_seq_len + 1) // 2) + 1]['mean'] = mean
                    metrics[idx - ((frames_seq_len + 1) // 2) + 1]['hist_mean'] = hist_mean

                    hist_mean_1q = float(np.nanmean(hist_metric_values_1q[-frames_seq_len: idx - ((frames_seq_len + 1) // 2) + 1] + hist_metric_values_1q[idx - ((frames_seq_len + 1) // 2) + 2:]))
                    hist_mean_2q = float(np.nanmean(hist_metric_values_2q[-frames_seq_len: idx - ((frames_seq_len + 1) // 2) + 1] + hist_metric_values_2q[idx - ((frames_seq_len + 1) // 2) + 2:]))
                    hist_mean_3q = float(np.nanmean(hist_metric_values_3q[-frames_seq_len: idx - ((frames_seq_len + 1) // 2) + 1] + hist_metric_values_3q[idx - ((frames_seq_len + 1) // 2) + 2:]))
                    hist_mean_4q = float(np.nanmean(hist_metric_values_4q[-frames_seq_len: idx - ((frames_seq_len + 1) // 2) + 1] + hist_metric_values_4q[idx - ((frames_seq_len + 1) // 2) + 2:]))

                    metrics[idx - ((frames_seq_len + 1) // 2) + 1]['hist_mean_1q'] = hist_mean_1q
                    metrics[idx - ((frames_seq_len + 1) // 2) + 1]['hist_mean_2q'] = hist_mean_2q
                    metrics[idx - ((frames_seq_len + 1) // 2) + 1]['hist_mean_3q'] = hist_mean_3q
                    metrics[idx - ((frames_seq_len + 1) // 2) + 1]['hist_mean_4q'] = hist_mean_4q

                elif idx == frames_seq_len - 2:
                    for i in range(0, (frames_seq_len) // 2):
                        mean = float(np.nanmean(metric_values[0: i] + metric_values[i + 1 : (frames_seq_len)// 2 + i + 1]))
                        metrics[i]['mean'] = mean
                        hist_mean = float(np.nanmean(hist_metric_values[0: i] + hist_metric_values[i + 1 : (frames_seq_len)// 2 + i + 1]))
                        metrics[i]['hist_mean'] = hist_mean

                        hist_mean_1q = float(np.nanmean(hist_metric_values_1q[0: i] + hist_metric_values_1q[i + 1 : (frames_seq_len)// 2 + i + 1]))
                        hist_mean_2q = float(np.nanmean(hist_metric_values_2q[0: i] + hist_metric_values_2q[i + 1 : (frames_seq_len)// 2 + i + 1]))
                        hist_mean_3q = float(np.nanmean(hist_metric_values_3q[0: i] + hist_metric_values_3q[i + 1 : (frames_seq_len)// 2 + i + 1]))
                        hist_mean_4q = float(np.nanmean(hist_metric_values_4q[0: i] + hist_metric_values_4q[i + 1 : (frames_seq_len)// 2 + i + 1]))

                        metrics[i]['hist_mean_1q'] = hist_mean_1q
                        metrics[i]['hist_mean_2q'] = hist_mean_2q
                        metrics[i]['hist_mean_3q'] = hist_mean_3q
                        metrics[i]['hist_mean_4q'] = hist_mean_4q

            else:
                frame_features['br_mean_prev'] = 0

                frame_features['metric_value'] = 0
                metric_values.append(0)
                frame_features['hist_metric_value'] = 0
                hist_metric_values.append(0)
                #frame_features['sobel_metric_value'] = 0

                frame_features['hist_metric_value_1q'] =  0
                frame_features['hist_metric_value_2q'] =  0
                frame_features['hist_metric_value_3q'] =  0
                frame_features['hist_metric_value_4q'] =  0

                hist_metric_values_1q.append(0)
                hist_metric_values_2q.append(0)
                hist_metric_values_3q.append(0)
                hist_metric_values_4q.append(0)

            prev_frame = frame
            prev_frame_hist = frame_hist
            prev_frame_sobel = frame_sobel

            prev_frame_1q_hist = frame_1q_hist
            prev_frame_2q_hist = frame_2q_hist
            prev_frame_3q_hist = frame_3q_hist
            prev_frame_4q_hist = frame_4q_hist

            metrics.append(frame_features)
            ###  END CODE HERE  ###
        for i in range(len(metric_values) - (frames_seq_len) // 2, len(metric_values)):
            mean = float(np.nanmean(metric_values[i - (frames_seq_len) // 2 : i] + metric_values[i + 1 : len(metric_values)]))
            metrics[i]['mean'] = mean
            hist_mean = float(np.nanmean(hist_metric_values[i - (frames_seq_len) // 2 : i] + hist_metric_values[i + 1 : len(metric_values)]))
            metrics[i]['hist_mean'] = hist_mean

            hist_mean_1q = float(np.nanmean(hist_metric_values_1q[i - (frames_seq_len) // 2 : i] + hist_metric_values_1q[i + 1 : len(metric_values)]))
            hist_mean_2q = float(np.nanmean(hist_metric_values_2q[i - (frames_seq_len) // 2 : i] + hist_metric_values_2q[i + 1 : len(metric_values)]))
            hist_mean_3q = float(np.nanmean(hist_metric_values_3q[i - (frames_seq_len) // 2 : i] + hist_metric_values_3q[i + 1 : len(metric_values)]))
            hist_mean_4q = float(np.nanmean(hist_metric_values_4q[i - (frames_seq_len) // 2 : i] + hist_metric_values_4q[i + 1 : len(metric_values)]))

            metrics[i]['hist_mean_1q'] = hist_mean_1q
            metrics[i]['hist_mean_2q'] = hist_mean_2q
            metrics[i]['hist_mean_3q'] = hist_mean_3q
            metrics[i]['hist_mean_4q'] = hist_mean_4q

        return metrics

    X_test = pd.DataFrame(metric_generator(frames))
    model = pickle.load(open("model.pkl", 'rb'))
    predict_cuts = model.predict(X_test)

    return np.where(predict_cuts == 1)[0], None, None

In [ ]:
video_dataset_path = 'train_dataset'
#Протестируем разработанный вами метод
run_scene_change_detector_all_video_dissolve(scene_change_detector_dissolve, video_dataset_path)

### Ваши ощущения ##

*Как и в первой части интересно узнать какие моменты показались простыми, а какие сложными. Много ли времени ушло на изучение sklearn и методов машинного обучения. Дало ли машинное обучение сразу прирост по сравнению с эврестической частью? Как вы контролировали переобучение?*